## 丢弃法（Dropout）— 从0开始
前面我们介绍了多层神经网络，就是包含至少一个隐含层的网络。我们也介绍了正则法来应对过拟合问题。在深度学习中，一个常用的应对过拟合问题的方法叫做丢弃法（Dropout）。本节以多层神经网络为例，从0开始介绍丢弃法。

由于丢弃法的概念和实现非常容易，在本节中，我们先介绍丢弃法的概念以及它在现代神经网络中是如何实现的。然后我们一起探讨丢弃法的本质。

### 丢弃法的概念
在现代神经网络中，我们所指的丢弃法，通常是对输入层或者隐含层做以下操作：

随机选择一部分该层的输出作为丢弃元素；
把丢弃元素乘以0；
把非丢弃元素拉伸。
### 丢弃法的实现
丢弃法的实现很容易，例如像下面这样。这里的标量`drop_probability`定义了一个`X`（`NDArray`类）中任何一个元素被丢弃的概率。

In [1]:
import numpy as np

def np_dropout(X, drop_probability):
    keep_probability = 1 - drop_probability
    assert 0 <= keep_probability <= 1
    # 这种情况下把全部元素都丢弃。
    if keep_probability == 0:
        return np.zeros(X.shape)

    # 随机选择一部分该层的输出作为丢弃元素。
    mask = np.random.uniform(0, 1.0, X.shape) < keep_probability
    # 保证 E[dropout(X)] == X
    scale =  1 / keep_probability
    return mask * X * scale, mask

def tf_dropout(X, drop_probability):

    keep_probability = 1 - drop_probability
    shape = X.get_shape().as_list()
    assert 0 <= keep_probability <= 1
    # 这种情况下把全部元素都丢弃。
    if keep_probability == 0:
        return tf.zeros(shape)

    # 随机选择一部分该层的输出作为丢弃元素。
    mask = tf.to_float(np.random.uniform(0, 1.0, shape) < keep_probability)
    # 保证 E[dropout(X)] == X
    scale =  1 / keep_probability
    return mask * X * scale

In [2]:
A = np.arange(20).reshape((5,4))
np_dropout(A, 0.0)

(array([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.],
        [16., 17., 18., 19.]]), array([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]))

In [3]:
np_dropout(A, 0.5)


(array([[ 0.,  0.,  4.,  0.],
        [ 0., 10.,  0.,  0.],
        [16., 18.,  0., 22.],
        [ 0., 26., 28., 30.],
        [32., 34.,  0.,  0.]]), array([[False, False,  True, False],
        [False,  True, False, False],
        [ True,  True, False,  True],
        [False,  True,  True,  True],
        [ True,  True, False, False]]))

In [4]:
np_dropout(A, 1.0)


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

### 丢弃法的本质
了解了丢弃法的概念与实现，那你可能对它的本质产生了好奇。

如果你了解集成学习，你可能知道它在提升弱分类器准确率上的威力。一般来说，在集成学习里，我们可以对训练数据集有放回地采样若干次并分别训练若干个不同的分类器；测试时，把这些分类器的结果集成一下作为最终分类结果。

事实上，丢弃法在模拟集成学习。试想，一个使用了丢弃法的多层神经网络本质上是原始网络的子集（节点和边）。举个例子，它可能长这个样子。
![image.png](http://zh.gluon.ai/_images/dropout.png)
我们在之前的章节里介绍过随机梯度下降算法：我们在训练神经网络模型时一般随机采样一个批量的训练数 据。丢弃法实质上是对每一个这样的数据集分别训练一个原神经网络子集的分类器。与一般的集成学习不同，这里每个原神经网络子集的分类器用的是同一套参数。因此丢弃法只是在模拟集成学习。

我们刚刚强调了，原神经网络子集的分类器在不同的训练数据批量上训练并使用同一套参数。因此，使用丢弃法的神经网络实质上是对输入层和隐含层的参数做了正则化：学到的参数使得原神经网络不同子集在训练数据上都尽可能表现良好。

下面我们动手实现一下在多层神经网络里加丢弃层。



### 数据获取
我们继续使用FashionMNIST数据集。


In [5]:
import sys

sys.path.append('../utils')
import utils

data_dir = '../data/fashion_mnist'
train_images, train_labels, test_images, test_labels = utils.load_data_fashion_mnist(data_dir, one_hot=True)
print train_images.shape
print train_labels.shape

from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
train_dataset = DataSet(train_images, train_labels, one_hot=True)

Extracting ../data/fashion_mnist/train-images-idx3-ubyte.gz
Extracting ../data/fashion_mnist/train-labels-idx1-ubyte.gz
Extracting ../data/fashion_mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/fashion_mnist/t10k-labels-idx1-ubyte.gz
(60000, 28, 28, 1)
(60000, 10)


### 含两个隐藏层的多层感知机
多层感知机已经在之前章节里介绍。与之前章节不同，这里我们定义一个包含两个隐含层的模型，两个隐含层都输出256个节点。我们定义激活函数Relu并直接使用Gluon提供的交叉熵损失函数。



In [6]:
import tensorflow as tf
num_inputs = 28*28
num_outputs = 10

num_hidden1 = 256
num_hidden2 = 256
weight_scale = .1

with tf.name_scope('multi_layer_percetron'):
    W1 = tf.Variable(tf.random_normal([num_inputs, num_hidden1], mean=0.0, stddev=weight_scale, seed=None, dtype=tf.float32), name='weights_hidden1')
    b1 = tf.Variable(tf.constant(0.0, shape=[num_hidden1]), name='bias_hidden')
    
    W2 = tf.Variable(tf.random_normal([num_hidden1, num_hidden2], mean=0.0, stddev=weight_scale, seed=None, dtype=tf.float32), name='weights_hidden2')
    b2 = tf.Variable(tf.constant(0.0, shape=[num_hidden2]), name='bias_output')
    
    W3 = tf.Variable(tf.random_normal([num_hidden2, num_outputs], mean=0.0, stddev=weight_scale, seed=None, dtype=tf.float32), name='weights_output')
    b3 = tf.Variable(tf.constant(0.0, shape=[num_outputs]), name='bias_output')

params = [W1, b1, W2, b2, W3, b3]


### 定义包含丢弃层的模型
我们的模型就是将层（全连接）和激活函数（Relu）串起来，并在应用激活函数后添加丢弃层。每个丢弃层的元素丢弃概率可以分别设置。一般情况下，我们推荐把更靠近输入层的元素丢弃概率设的更小一点。这个试验中，我们把第一层全连接后的元素丢弃概率设为0.2，把第二层全连接后的元素丢弃概率设为0.5。

In [7]:
drop_prob1 = 0.2
drop_prob2 = 0.8

def net(X):
    is_training = True
    # 第一层全连接。
    h1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    # 在第一层全连接后添加丢弃层。
    #h1 = tf_dropout(h1, drop_prob1)
    h1 = tf.map_fn(lambda h: tf_dropout(h, drop_prob1), h1)
    # 第二层全连接。
    h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
    # 在第二层全连接后添加丢弃层。
    h2 = tf.map_fn(lambda h: tf_dropout(h, drop_prob2), h2)
    #h2 = tf_dropout(h2, drop_prob2)
    return tf.matmul(h2, W3) + b3

### 训练
训练跟之前一样。



In [8]:
learning_rate = 1e-2
max_steps = 10000
batch_size = 32
train_loss = 0.0
train_acc = 0.0

input_placeholder = tf.placeholder(tf.float32, [None, num_inputs])
gt_placeholder = tf.placeholder(tf.int64, [None, num_outputs])
logits = net(input_placeholder)
loss = tf.losses.softmax_cross_entropy(logits=logits,  onehot_labels=gt_placeholder)

acc = utils.accuracy(logits , gt_placeholder)
var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


for step in range(max_steps):
    data, label = train_dataset.next_batch(batch_size)
    data = np.reshape(data, (batch_size, num_inputs))
    feed_dict = {input_placeholder: data.reshape((-1, num_inputs)), gt_placeholder: label}
    loss_, acc_, _ = sess.run([loss, acc, train_op], feed_dict=feed_dict)
    if step % 1000 == 0:
        print 'step %d, train loss %f' % (step, loss_)
        print 'step %d, train acc %f' % (step, acc_)
test_acc = sess.run(acc, feed_dict={input_placeholder: np.squeeze(test_images).reshape((-1, num_inputs)) / 255.0 , gt_placeholder: test_labels})
print 'step %d, test acc %f' % (step, test_acc)



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

multi_layer_percetron/weights_hidden1
multi_layer_percetron/bias_hidden
multi_layer_percetron/weights_hidden2
multi_layer_percetron/bias_output
multi_layer_percetron/weights_output
multi_layer_percetron/bias_output_1
step 0, train loss 4.047252
step 0, train acc 0.093750
step 1000, train loss 0.756845
step 1000, train acc 0.750000
step 2000, train loss 0.389689
step 2000, train acc 0.875000
step 3000, train loss 0.272325
step 3000, train acc 0.875000
step 4000, train loss 0.428824
step 4000, train acc 0.781250
step 5000, train loss 0.363320
step 5000, train acc 0.843750
step 6000, train loss 0.305298
step 6000, train acc 0.875000
step 7000, train loss 0.329539
step 7000, train acc 0.906250
step 8000, train loss 0.275392
step 8000, train acc 0.906250
step 9000, train loss 0.615244
step 9000, train acc